# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [11]:
# Import required libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display

In [32]:
import gradio as gr # oh yeah!

In [13]:
# Load environment variables from .env file
load_dotenv(override=True)

# API keys from environment
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_model = os.getenv('GOOGLE_MODEL')
ollama_base_url = os.getenv('OLLAMA_BASE_URL')
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_model = os.getenv('OLLAMA_MODEL', 'deepseek-v3.1:671b-cloud')

# Verify API keys
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[:7]}...")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[:2]}...")
else:
    print("Google API Key not set")

if ollama_base_url:
    print(f"Ollama configured at: {ollama_base_url}")

OpenAI API Key loaded: sk-proj-...
Anthropic API Key loaded: sk-ant-...
Google API Key loaded: AI...
Ollama configured at: http://192.168.80.200:11434


In [65]:
# Initialize API clients
openai_client = OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=anthropic_api_key)
google.generativeai.configure(api_key=google_api_key)

# Initialize Ollama client (uses OpenAI-compatible API)
ollama_client = OpenAI(
    base_url=f"{ollama_base_url}/v1",
    api_key=ollama_api_key
)

print("All clients initialized successfully")

All clients initialized successfully


In [74]:
def call_ollama(prompt, stream=False):
    """Call Ollama model with OpenAI-compatible API"""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    response = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream=stream
    )
    
    if stream:
        for chunk in response:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end='', flush=True)
    else:
        return response.choices[0].message.content

In [75]:
# Message generic system
system_message = "You are a helpful assistant"
prompt= "What is today's date?"

In [76]:
# This can reveal the "training cut off", or the most recent date in the training data
call_ollama(prompt, stream=False)

"I'm sorry, but I cannot access real-time data or the current date. Please check your device's settings or a reliable online source for today's date. Let me know if there's anything else I can help with! 😊"

## User Interface time!

**IMPORTANTE para Docker/JupyterLab:**
- Gradio no se muestra inline, debes abrir manualmente en el navegador
- Usa diferentes puertos para cada demo (7860, 7861, 7862, etc.)
- Accede a: `http://192.168.80.200:PUERTO`
- Cierra las demos anteriores con `gr.close_all()` si necesitas reutilizar el puerto

In [18]:
# here's a simple function

def shout(text):
    #print(f"Shout has been called with input {text}")
    return text.upper()

In [19]:
shout("hello")

'HELLO'

In [37]:
# Cerrar todas las instancias previas de Gradio
gr.close_all()

demo = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never")
demo.launch(share=True)
#demo.launch()

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://76d25c84a684426430.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [40]:
# Cerrar todas las instancias previas de Gradio y abre en un navegador independiente gradio con .launch(inbrowser=True)
gr.close_all()

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True, inbrowser=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://28773454e7f2399bce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Adding inbrowser=True opens up a new browser window automatically
gr.close_all()
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Adding authentication

Gradio makes it very easy to have userids and passwords

Obviously if you use this, have it look properly in a secure place for passwords! At a minimum, use your .env

In [48]:
gr.close_all()
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True, inbrowser=True, auth=("ed", "bananas"))

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://6b668aad8fc33ba41d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [46]:
# Define this variable and then pass js=force_dark_mode when creating the Interface
gr.close_all()

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://509287555c4dbde9d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [77]:
# Adding a little more: with call Ollama
gr.close_all()

view = gr.Interface(
    fn=call_ollama,
    title="Shout", 
    inputs=[gr.Textbox(label="Your message", lines=6)], 
    outputs=[gr.Textbox(label="Response", lines=8)],
    flagging_mode="never"
    )
view.launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://fd4ee113e60a0ffd26.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [73]:
# Adding a little more: with call Ollama
gr.close_all()

system_message = "Eres un asistente util que responde en formato markdown"

view = gr.Interface(
    fn=call_ollama,
    title="Shout", 
    inputs=[gr.Textbox(label="Your message")], 
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never"
    )
view.launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2a4aaf5bdcd9675266.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [88]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python guide in the guides folder

system_message = "Eres un asistente util que responde en formato markdown"

def call_ollama(prompt):
    """Call Ollama model with OpenAI-compatible API"""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    stream = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream = True
    )
    result=""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [89]:
# Adding a little more: with call Ollama
gr.close_all()

system_message = "Eres un asistente util que responde en formato markdown"

view = gr.Interface(
    fn=call_ollama,
    title="Shout", 
    inputs=[gr.Textbox(label="Your message")], 
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never"
    )
view.launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://1c6d6cb7f18d064640.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## And now getting fancy

Remember to check the Intermediate Python Guide if you're unsure about generators and "yield"

In [90]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [92]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")
gr.close_all()

view = gr.Interface(
    fn=stream_model,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch(share=True)

Closing server running on port: 7860
Closing server running on port: 7861
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3002133fe6b8d9da49.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Building a company brochure generator

Now you know how - it's simple!

In [ ]:
from scraper import fetch_website_contents

In [ ]:

# Again this is typical Experimental mindset - I'm changing the global variable we used above:

system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [ ]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [96]:
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL including http:// or https://")
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")
gr.close_all()

view = gr.Interface(
    fn=stream_model,
    title="Brochure Generator", 
    inputs=[name_input, url_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Hugging Face", "https://huggingface.co", "GPT"],
            ["Edward Donner", "https://edwarddonner.com", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch(share=True)

/opt/conda/envs/LLM/lib/python3.11/site-packages/gradio/utils.py:1052: UserWarning: Expected 2 arguments for function <function stream_model at 0x7f7641b227a0>, received 3.
  warnings.warn(
/opt/conda/envs/LLM/lib/python3.11/site-packages/gradio/utils.py:1060: UserWarning: Expected maximum 2 arguments for function <function stream_model at 0x7f7641b227a0>, received 3.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://b3a572fb6b9f2fbdcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [97]:
gr.close_all()

Closing server running on port: 7860
